In [1]:
import pandas as pd
import os
from constants import *
import folium
import IPython
from folium.plugins import HeatMap
import glob
from folium.plugins import HeatMapWithTime
from jinja2 import Template
from folium.map import Layer
from tqdm.auto import tqdm

In [2]:
def generateBaseMap(default_location=[47.11126144, 8.4375], default_zoom_start=5):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start, tiles = "Stamen Toner")
    return base_map

In [3]:
class HeatMapWithTimeAdditional(Layer):
    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = new TDHeatmap({{ this.data }},
                {heatmapOptions: {
                    radius: {{this.radius}},
                    minOpacity: {{this.min_opacity}},
                    maxOpacity: {{this.max_opacity}},
                    scaleRadius: {{this.scale_radius}},
                    useLocalExtrema: {{this.use_local_extrema}},
                    defaultWeight: 1,
                    {% if this.gradient %}gradient: {{ this.gradient }}{% endif %}
                }
            }).addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, data, name=None, radius=15,
                 min_opacity=0, max_opacity=0.6,
                 scale_radius=False, gradient=None, use_local_extrema=False,
                 overlay=True, control=True, show=True):
        super(HeatMapWithTimeAdditional, self).__init__(
            name=name, overlay=overlay, control=control, show=show
        )
        self._name = 'HeatMap'
        self.data = data

        # Heatmap settings.
        self.radius = radius
        self.min_opacity = min_opacity
        self.max_opacity = max_opacity
        self.scale_radius = 'true' if scale_radius else 'false'
        self.use_local_extrema = 'true' if use_local_extrema else 'false'
        self.gradient = gradient

In [18]:
#LOAD AND CLEAN COVID DATA
df = pd.read_csv(os.path.join(DATA_PATH, "Bing-COVID-19-Data", "data", "Bing-COVID19-Data.csv")).fillna(0)
df_covid_copy = df[df.Country_Region != "Worldwide"].copy()
df_covid_copy = df_covid_copy[['Updated','Latitude', 'Longitude', 'ConfirmedChange']]
df_covid_copy["Updated"] = pd.to_datetime(df.Updated)
df_covid_copy = df_covid_copy[df_covid_copy.Updated < "2020-06-08"]

['/mnt/data/Bing-COVID-19-Data/data/COVID_Data_cleaned.csv']

In [22]:
#LOAD AND AGGREGATE TWITTER DATA
geo_tweets_file, place_tweets_file = glob.glob(TWITTER_PATH + "/*.csv")
geo_tweets = pd.read_csv(geo_tweets_file)
place_tweets = pd.read_csv(place_tweets_file)
place_tweets = place_tweets.rename(columns={"estimated_longitude": "Longitude", "estimated_latitude": "Latitude", "created_at": "Updated"})
geo_tweets = geo_tweets.rename(columns={"longitude": "Longitude", "latitude": "Latitude", "created_at": "Updated"})
all_tweets = pd.concat([place_tweets, geo_tweets])
all_tweets = all_tweets[['Updated', 'Longitude', 'Latitude']]
all_tweets["Updated"] = pd.to_datetime(all_tweets.Updated)
all_tweets_copy = all_tweets.copy()
#to have same time scale as covid data
all_tweets_copy = all_tweets_copy[all_tweets_copy.Updated > pd.to_datetime("2020-01-20")]

['/mnt/data/All_Tweets_cleaned.csv']

In [12]:
base_map = generateBaseMap()

In [13]:
#COVID CASES OVER TIME
df_date_list_cov = []
for date in df_covid_copy.Updated.sort_values().unique():
    df_date_list_cov.append(df_covid_copy.loc[df_covid_copy.Updated == date, ['Latitude', 'Longitude', 'ConfirmedChange']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

hmap_time_covid=HeatMapWithTime(df_date_list_cov, radius=20, min_opacity = 0.5, gradient ={0.2: 'yellow', 0.5: 'gold', 0.8: 'orange', 1.0: 'red'}, max_opacity=0.8, use_local_extrema=True, overlay=False, name="Covid").add_to(base_map)


In [8]:
#TWITTER CASES OVER TIME
df_date_list = []
for date in tqdm(all_tweets_copy.Updated.sort_values().unique()):
    df_date_list.append(all_tweets_copy.loc[all_tweets_copy.Updated == date, ['Latitude', 'Longitude']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [14]:
#hmap_time_twitter=HeatMapWithTime(df_date_list, radius=20, min_opacity = 0.5, gradient ={0.2: 'lightsteelblue', 0.5: 'cornflowerblue', 0.7: 'royalblue', 1.0: 'navy'}, max_opacity=0.8, use_local_extrema=True, overlay=True).add_to(base_map)
hmap_time_twitter = HeatMapWithTimeAdditional(df_date_list,overlay=False,name='Twitter',min_opacity=0.05,radius=20,max_opacity=1, gradient ={0.2: 'lightsteelblue', 0.5: 'cornflowerblue', 0.7: 'royalblue', 1.0: 'navy'}).add_to(base_map)



In [15]:
#ADD TO MAP
base_map.add_child(hmap_time_twitter)
base_map.add_child(hmap_time_covid)

In [ ]:
base_map